In [164]:
import numpy as np
import pandas as pd
from pathlib import Path

## Conformalized Quantile Regression

In [81]:
class QRErrorErrFunc:
    """Calculates conformalized quantile regression error.
    Conformity scores:
    .. math::
    max{\hat{q}_low - y, y - \hat{q}_high}
    """

    def __init__(self):
        super(QRErrorErrFunc, self).__init__()

    def apply(self, prediction, y):
        y_lower = prediction[:, 0]
        y_upper = prediction[:, -1]
        error_low = y_lower - y
        error_high = y - y_upper
        err = np.maximum(error_high, error_low)
        return err

    def apply_inverse(self, nc, alpha):
        q = np.quantile(
            nc, np.minimum(1.0, (1.0 - alpha) * (nc.shape[0] + 1.0) / nc.shape[0])
        )
        return np.vstack([q, q])
    
class AbsErrorErrFunc:
    def __init__(self):
        super(AbsErrorErrFunc, self).__init__()

    def apply(self,
              prediction,
              y):
        return np.abs(prediction[:, 0] - y) / (prediction[:, 1])

    def apply_inverse(self,
                      nc,
                      significance):
        q = np.quantile(
            nc, np.minimum(1.0, (1.0 - significance) * (nc.shape[0] + 1.0) / nc.shape[0])
        )
        return np.vstack([q, q])

In [7]:
def run_cqr_icp(y_preds_calib, y_true_calib, y_preds_test, alpha: float = 0.1):
    scorer = QRErrorErrFunc()
    nc = scorer.apply(y_preds_calib, y_true_calib)
    scores_correction = scorer.apply_inverse(nc, alpha)
    y_intervals = y_preds_test
    y_intervals[:, 0] -= scores_correction[0, 0]
    y_intervals[:, 1] += scores_correction[1, 0]
    return y_intervals

In [131]:
def run_abs_icp(y_preds_calib, y_true_calib, y_preds_test, alpha: float = 0.1):
    scorer = AbsErrorErrFunc()
    nc = scorer.apply(y_preds_calib, y_true_calib)
    scores_correction = scorer.apply_inverse(nc, alpha)
    y_intervals = np.zeros_like(y_preds_test)
    y_intervals[:, 0] = y_preds_test[:, 0] - scores_correction[0, 0] * y_preds_test[:, 1]
    y_intervals[:, 1] = y_preds_test[:, 0] + scores_correction[1, 0] * y_preds_test[:, 1]
    return y_intervals

In [173]:
def evaluate(pred, Y):
    # Extract lower and upper prediction bands
    pred_l = np.min(pred,1)
    pred_h = np.max(pred,1)
    # Marginal coverage
    cover = (Y>=pred_l)*(Y<=pred_h)
    marg_coverage = np.mean(cover)
    # if X is None:
    #     wsc_coverage = None
    # else:
    #     # Estimated conditional coverage (worse-case slab)
    #     wsc_coverage = coverage.wsc_unbiased(X, Y, pred, M=100)

    # Marginal length
    lengths = pred_h-pred_l
    length = np.mean(lengths)
    # Length conditional on coverage
    idx_cover = np.where(cover)[0]
    length_cover = np.mean([lengths[i] for i in idx_cover])

    # Combine results
    out = {'Coverage': marg_coverage, 'Length': length, 'Length cover': length_cover}
    return out

In [156]:
test_df = pd.read_csv("./experiments/HLM_Fang2023/IVIT/fold2/JQR/testset_predictions.csv")
val_df = pd.read_csv("./experiments/HLM_Fang2023/IVIT/fold2/JQR/valset_predictions.csv")

In [157]:
y_preds_calib = val_df.loc[:, ["Y_PRED_Q5", "Y_PRED_Q95"]].values
y_residuals_calib = val_df.loc[:, ["Y_POSTERIOR_MEAN", "Y_POSTERIOR_STDDEV"]].values
y_true_calib = val_df.loc[:, ["Y_TRUE"]].values
y_preds_test = test_df.loc[:, ["Y_PRED_Q5", "Y_PRED_Q95"]].values
y_residuals_test = test_df.loc[:, ["Y_POSTERIOR_MEAN", "Y_POSTERIOR_STDDEV"]].values
y_true_test = test_df.loc[:, ["Y_TRUE"]].values

In [158]:
y_preds_cqr = run_cqr_icp(y_preds_calib, y_true_calib, y_preds_test)

In [159]:
evaluate(y_preds_cqr, y_true_test)

,Coverage,Length,Length cover
0,0.89733,1.994593,1.964476


In [160]:
y_preds_res = run_abs_icp(y_residuals_calib, y_true_calib, y_residuals_test)

In [161]:
evaluate(y_preds_res, y_true_test)

,Coverage,Length,Length cover
0,0.902012,2.496254,2.453881


In [162]:
y_preds_ens = (y_preds_res + y_preds_cqr) / 2

In [163]:
evaluate(y_preds_ens, y_true_test)

,Coverage,Length,Length cover
0,0.901373,2.245424,2.209346


In [165]:
DATA_DIR = Path("./experiments/")

In [174]:
out = []
for filepath in DATA_DIR.rglob("testset_predictions.csv"):
    fields = str(filepath).split("/")
    dataset, split, fold, ue = fields[1:-1]
    val_df = pd.read_csv(filepath.parent / "valset_predictions.csv")
    test_df = pd.read_csv(filepath)
    y_preds_calib = val_df.loc[:, ["Y_PRED_Q5", "Y_PRED_Q95"]].values
    y_residuals_calib = val_df.loc[:, ["Y_POSTERIOR_MEAN", "Y_POSTERIOR_STDDEV"]].values
    y_true_calib = val_df.loc[:, ["Y_TRUE"]].values
    y_preds_test = test_df.loc[:, ["Y_PRED_Q5", "Y_PRED_Q95"]].values
    y_residuals_test = test_df.loc[:, ["Y_POSTERIOR_MEAN", "Y_POSTERIOR_STDDEV"]].values
    y_true_test = test_df.loc[:, ["Y_TRUE"]].values
    y_preds_cqr = run_cqr_icp(y_preds_calib, y_true_calib, y_preds_test)
    metrics = evaluate(y_preds_cqr, y_true_test)
    metrics["dataset"] = dataset
    metrics["split"] = split
    metrics["fold"] = fold
    metrics["ue"] = ue
    out.append(metrics)

In [175]:
cp_metrics = pd.DataFrame(out)

In [176]:
cp_metrics

,Coverage,Length,Length cover,dataset,split,fold,ue
0,0.892140,2.652361,2.649429,VDss_Liu2022,IVIT,fold0,JMQR
1,0.893512,2.581362,2.577170,VDss_Liu2022,IVIT,fold0,JQR
2,0.909042,2.635194,2.636295,VDss_Liu2022,IVIT,fold1,JMQR
3,0.910769,2.686149,2.685782,VDss_Liu2022,IVIT,fold1,JQR
4,0.903707,2.768918,2.762201,VDss_Liu2022,IVIT,fold2,JMQR
...,...,...,...,...,...,...,...
535,0.849595,8.977176,8.967183,Solubility_Wang2020,OVOT,fold7,JQR
536,0.943955,9.221580,9.214099,Solubility_Wang2020,OVOT,fold8,JMQR
537,0.947103,9.152488,9.139972,Solubility_Wang2020,OVOT,fold8,JQR
538,0.853723,9.098156,9.092947,Solubility_Wang2020,OVOT,fold9,JMQR


In [178]:
cp_metrics.groupby(['dataset', 'split', 'ue'])['Coverage'].apply(lambda x: f"{np.mean(x):.3f} ({np.std(x):.3f})").reset_index().pivot(index=['dataset', 'ue'], columns='split', values='Coverage')

split                                      IVIT           IVOT           OVOT
dataset                     ue                                               
HLM_Fang2023                JMQR  0.903 (0.015)  0.909 (0.008)  0.896 (0.017)
                            JQR   0.902 (0.020)  0.906 (0.009)  0.900 (0.015)
LD50_Lunghini2019           JMQR  0.901 (0.006)  0.921 (0.013)  0.901 (0.031)
                            JQR   0.901 (0.006)  0.921 (0.013)  0.896 (0.033)
Lipophilicity_Wang2020      JMQR  0.902 (0.005)  0.902 (0.008)  0.902 (0.007)
                            JQR   0.902 (0.005)  0.901 (0.007)  0.902 (0.007)
Permeability_Caco2_Wang2020 JMQR  0.902 (0.003)  0.906 (0.008)  0.900 (0.011)
                            JQR   0.902 (0.002)  0.903 (0.008)  0.902 (0.010)
Permeability_MDCK_Fang2023  JMQR  0.903 (0.006)  0.901 (0.010)  0.903 (0.015)
                            JQR   0.902 (0.006)  0.901 (0.009)  0.902 (0.016)
RLM_Fang2023                JMQR  0.901 (0.006)  0.904 (0.006)  0.902 (0.010)
                            JQR   0.909 (0.016)  0.905 (0.005)  0.902 (0.008)
Solubility_Wang2020         JMQR  0.901 (0.003)  0.910 (0.024)  0.902 (0.035)
                            JQR   0.901 (0.003)  0.911 (0.025)  0.895 (0.035)
VDss_Liu2022                JMQR  0.904 (0.006)  0.904 (0.010)  0.902 (0.014)
                            JQR   0.904 (0.006)  0.903 (0.011)  0.907 (0.011)
hPPB_Lou2022                JMQR  0.901 (0.006)  0.905 (0.021)  0.896 (0.023)
                            JQR   0.903 (0.006)  0.904 (0.020)  0.899 (0.019)

In [179]:
cp_metrics.groupby(['dataset', 'split', 'ue'])['Length cover'].apply(lambda x: f"{np.mean(x):.3f} ({np.std(x):.3f})").reset_index().pivot(index=['dataset', 'ue'], columns='split', values='Length cover')

split                                      IVIT           IVOT           OVOT
dataset                     ue                                               
HLM_Fang2023                JMQR  2.080 (0.122)  2.152 (0.152)  2.089 (0.084)
                            JQR   2.043 (0.119)  2.070 (0.167)  2.065 (0.084)
LD50_Lunghini2019           JMQR  3.855 (0.098)  3.910 (0.515)  3.332 (0.497)
                            JQR   3.828 (0.081)  3.868 (0.527)  3.340 (0.579)
Lipophilicity_Wang2020      JMQR  5.156 (0.196)  5.126 (0.361)  4.986 (0.352)
                            JQR   5.126 (0.173)  5.079 (0.313)  4.929 (0.363)
Permeability_Caco2_Wang2020 JMQR  2.964 (0.123)  2.942 (0.166)  2.637 (0.175)
                            JQR   2.903 (0.130)  2.883 (0.235)  2.644 (0.170)
Permeability_MDCK_Fang2023  JMQR  2.621 (0.148)  2.541 (0.172)  2.429 (0.150)
                            JQR   2.596 (0.173)  2.533 (0.171)  2.420 (0.158)
RLM_Fang2023                JMQR  2.987 (0.067)  3.034 (0.119)  3.047 (0.123)
                            JQR   2.871 (0.211)  3.069 (0.110)  3.039 (0.103)
Solubility_Wang2020         JMQR  9.310 (0.179)  9.046 (1.006)  8.632 (1.091)
                            JQR   9.286 (0.188)  9.074 (0.966)  8.320 (1.168)
VDss_Liu2022                JMQR  2.704 (0.116)  2.639 (0.202)  2.480 (0.202)
                            JQR   2.673 (0.104)  2.615 (0.213)  2.478 (0.213)
hPPB_Lou2022                JMQR  1.301 (0.116)  1.240 (0.181)  1.045 (0.140)
                            JQR   1.273 (0.082)  1.262 (0.199)  1.051 (0.120)